1. Setup Pyspark

In [1]:
# Insttall Pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 48 kB/s 
     |████████████████████████████████| 199 kB 45.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=61eb8d28eeab59eda770c152e0d7527abcd7325648ce074cfd9a1ee6c9582bda
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
# Import SparkSession
from pyspark.sql import SparkSession 

In [4]:
# Konfigurasi Spark

# Membuat SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("uts")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Deklarasi fungsi
sc = spark.sparkContext

2.	Baca data dari FullData.csv, kemudian pecah setidaknya menjadi 2 RDD (players, clubs) 

In [5]:
# Read CSV file atau membaca data
df = spark.read.csv("FullData.csv", header=True, inferSchema=True)

df.printSchema() # Berfungsi menampilkan tipe data dati setiap kolom yang ada
df.show(10)

root
 |-- Name: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- National_Position: string (nullable = true)
 |-- National_Kit: double (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club_Position: string (nullable = true)
 |-- Club_Kit: double (nullable = true)
 |-- Club_Joining: string (nullable = true)
 |-- Contract_Expiry: double (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Preffered_Foot: string (nullable = true)
 |-- Birth_Date: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Preffered_Position: string (nullable = true)
 |-- Work_Rate: string (nullable = true)
 |-- Weak_foot: integer (nullable = true)
 |-- Skill_Moves: integer (nullable = true)
 |-- Ball_Control: integer (nullable = true)
 |-- Dribbling: integer (nullable = true)
 |-- Marking: integer (nullable = true)
 |-- Sliding_Tackle: integer (nullable = true)
 |-- Standing_Tackle:

In [6]:
# Membagi file menjadi 2 RDD

# Drop kolom Club
Players_RDD = df.drop('Club').rdd

# Menggunakan kolom Club
Club_RDD = df.select('Club').distinct().rdd

# 3. Mengubah RDD menjadi PySpark Dataframe

In [7]:
# Mengubah RDD menjadi DataFrame
Players_df = Players_RDD.toDF()
Club_df = Club_RDD.toDF()

# 4. Agregasi jumlah pemain masing-masing kelompok berdasarkan:

## a. Nationaly

## b. Club

In [8]:
# Agregasi pemain berdasarkan 'Nationaly'
from pyspark.sql import functions as C

Players_count = Players_df.groupBy('Nationality').agg(C.count('Name')\
  .alias('Players_Agg')).orderBy('Players_Agg', ascending=False) 

Players_count.show(10)

+-------------------+-----------+
|        Nationality|Players_Agg|
+-------------------+-----------+
|            England|       1618|
|          Argentina|       1097|
|              Spain|       1008|
|             France|        974|
|             Brazil|        921|
|              Italy|        751|
|            Germany|        689|
|           Colombia|        592|
|              Japan|        471|
|Republic of Ireland|        442|
+-------------------+-----------+
only showing top 10 rows



In [9]:
# Agregasi pemain berdasarkan 'Club'
Club_count = Club_df.join(df, ['Club'], "inner")
Club_count.groupBy('Club').agg(C.count('Name').alias('Players_Agg')).show(10)

+---------------+-----------+
|           Club|Players_Agg|
+---------------+-----------+
|        Palermo|         28|
|Shonan Bellmare|         24|
|    Yeovil Town|         24|
|     Sagan Tosu|         25|
|          Carpi|         26|
| Kaiserslautern|         28|
|   Sparta R'dam|         29|
|       FC Basel|         27|
|  Karlsruher SC|         28|
|Cheltenham Town|         25|
+---------------+-----------+
only showing top 10 rows



# 5. Agregasi nilai:

### a. Rata-rata rating berdasarkan Club_Position

### b. Max rating berdasarkan National_Position

In [10]:
# a. Agregasi rating berdasakan Club_Position
Club_Position_avg_rating = Players_df.groupBy('Club_Position')\
.agg(C.avg('Rating').alias('Club_Position_avg_rating'))

Club_Position_avg_rating.show(10)

+-------------+------------------------+
|Club_Position|Club_Position_avg_rating|
+-------------+------------------------+
|           RF|                    72.0|
|          LWB|       67.44444444444444|
|          LCM|       68.77966101694915|
|           LM|       68.81884057971014|
|          RDM|       69.81954887218045|
|           LF|       71.08333333333333|
|         null|                    81.0|
|          CAM|        70.5904761904762|
|          RAM|       71.22222222222223|
|           LB|       68.10382513661202|
+-------------+------------------------+
only showing top 10 rows



In [11]:
# b. Agregasi max rating berdasakan National_Position
National_Position_maxrating = Players_df.groupBy('National_Position')\
.agg(C.max('Rating').alias('National_Position_Max_Rating'))

National_Position_maxrating.show(10)

+-----------------+----------------------------+
|National_Position|National_Position_Max_Rating|
+-----------------+----------------------------+
|               RF|                          85|
|              LWB|                          75|
|              LCM|                          88|
|               LM|                          87|
|              RDM|                          84|
|               LF|                          89|
|             null|                          90|
|              CAM|                          89|
|              RAM|                          83|
|               LB|                          86|
+-----------------+----------------------------+
only showing top 10 rows



# 6. Melakukan 3 Operasi transform RDD

### a. ReduceByKey

### b. ShortByKey

### c. Filtering

In [12]:
# a. ReduceByKey
Club_kit = df.select('Club', 'Club_Kit').rdd 
Club_sum_map = Club_kit.map(lambda row: (row.Club, row.Club_Kit))

In [13]:
Club_kit_sum = Club_sum_map.reduceByKey(lambda x, y: x + y)
for key, value in Club_kit_sum.take(10): print(f'{key} has a club kit sum of {value}')

Real Madrid has a club kit sum of 582.0
FC Barcelona has a club kit sum of 640.0
FC Bayern has a club kit sum of 521.0
Manchester Utd has a club kit sum of 814.0
Chelsea has a club kit sum of 868.0
Arsenal has a club kit sum of 766.0
Juventus has a club kit sum of 633.0
PSG has a club kit sum of 576.0
Manchester City has a club kit sum of 1069.0
Atlético Madrid has a club kit sum of 567.0


In [14]:
#b. ShortbyKey
swap_Club_kit_sum = Club_kit_sum.map( lambda row: (row[1], row[0]))

In [15]:
sorted_kit_sum = swap_Club_kit_sum.sortByKey(ascending=False)
for key, value in Club_kit_sum.take(10): print(f'Sum rating of {value} is from {key} club')

Sum rating of 582.0 is from Real Madrid club
Sum rating of 640.0 is from FC Barcelona club
Sum rating of 521.0 is from FC Bayern club
Sum rating of 814.0 is from Manchester Utd club
Sum rating of 868.0 is from Chelsea club
Sum rating of 766.0 is from Arsenal club
Sum rating of 633.0 is from Juventus club
Sum rating of 576.0 is from PSG club
Sum rating of 1069.0 is from Manchester City club
Sum rating of 567.0 is from Atlético Madrid club


In [16]:
# c. Filtering Goal Keeper (GK) terbaik di setiap club
GK_filtering = Players_RDD.filter(
lambda Filtering_Players: Filtering_Players.Club_Position == 'GK' 
  and Filtering_Players.Rating >= 89)

for x in GK_filtering.collect():
  print(f'Posisi {x.Club_Position} dengan rating {x.Rating} dimiliki oleh {x.Name}')

Posisi GK dengan rating 92 dimiliki oleh Manuel Neuer
Posisi GK dengan rating 90 dimiliki oleh De Gea
Posisi GK dengan rating 89 dimiliki oleh Thibaut Courtois
